In [1]:
# import packages
import pandas as pd
import numpy as np
import re, requests, math

from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

In [2]:
# input user_id, user_agent and cookie
name = 'np1221'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15'
cookie = 'dbcl2="60179733:aQrTO5OjX14"; _pk_ses.100001.3ac3=1; __utma=81379588.1499019577.1721712750.1744728106.1745419849.53; __utmb=81379588.1.10.1745419849; __utmc=81379588; __utmz=81379588.1725819519.15.3.utmcsr=search.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/book/subject_search; __utma=30149280.1824874719.1721712750.1744728106.1745419849.54; __utmb=30149280.1.10.1745419849; __utmc=30149280; __utmv=30149280.6017; __utmz=30149280.1722836274.6.2.utmcsr=book.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/mine; _vwo_uuid_v2=DECC1286962E715F2E5889CE127727FFB|471e8136287413b43e5b191b16f99c17; push_doumail_num=0; push_noty_num=0; _pk_ref.100001.3ac3=%5B%22%22%2C%22%22%2C1745419848%2C%22https%3A%2F%2Fsearch.douban.com%2Fbook%2Fsubject_search%3Fsearch_text%3D%E5%A4%96%E5%9B%BD%E6%82%AC%E7%96%91%E6%95%85%E4%BA%8B%26cat%3D1001%26start%3D15%22%5D; _pk_id.100001.3ac3=c8e7b6af39c07231.1721712750.; ct=y; viewed="37096828"; _vwo_uuid_v2=DECC1286962E715F2E5889CE127727FFB|471e8136287413b43e5b191b16f99c17; FCNEC=%5B%5B%22AKsRol9KPgWpMAhLLAzTh3LeiW9VUcVNsqI2ukWH_kitHXzG-pFPI7UbIf2kI7eCyBDmUXR3xlZQVXl1nNSCh-HtcP31Dr4PUF5zLBtNC-wtkezYScV7vJnk4GYjmx0fbUnKzQVe2xYAcIgZ97sqOfwr-pmilERP1w%3D%3D%22%5D%5D; __gads=ID=f6193f9ba868de93:T=1721713008:RT=1727077713:S=ALNI_Maf645eieU3fi5J7aN4iaUefmmxxg; __gpi=UID=00000e8969053eb6:T=1721713008:RT=1727077713:S=ALNI_MZuqnbwAqCLaaVaAYlVujNujwGQfA; douban-fav-remind=1; _ga=GA1.1.265002273.1722836289; _ga_KY2XQG4Q00=GS1.1.1724430590.6.1.1724431007.60.0.1879545909; bid=7WXEQNkOcG0; __yadk_uid=aUZvPUPiw7968NeqOexwitkHzKMsPYLt'

# generate the header
header = {'User-Agent': user_agent, 'Cookie': cookie}

In [3]:
# define the function of parse each book in one page
def get_books_info_from_page(url, header):
    
    # create the dataframe for all used columns
    page_data = pd.DataFrame(columns = ['Name', 'Pub_info', 'Date', 'Star', 'Tag', 'Comment', 'Website'])
    
    # get the web data from the url
    web_data = requests.get(url, headers = header)
    soup = BeautifulSoup(web_data.text, features = "lxml")

    # parse by each book
    for div in soup.select('div.info'):
        if div.find('a').contents[0] == '读书主页':
            pass
        else:
            try:
                # get the feature of each book
                book_name = str(div.find('a').contents[0]).replace('  ','').replace('\n','')
                book_pubinfo = str(div.select('div.pub')).split('>')[1].split('<')[0].replace('  ','').replace('\n','')
                book_date = str(div.select('span.date')).split('>')[1].split('<')[0][:10]
                book_comment = str(div.find('p')).split('>')[1].split('<')[0].replace('  ','').replace('\n','')
                book_tag = str(div.select('span.tags')).split('>')[1].split('<')[0][4:]
                book_star = [i for i in range(1, 6) if div.select(f'span.rating{i}-t')][0]
                book_website = re.search(r'href="([^"]+)"', str(div.select('a')[0])).group(1)
                
                # combine the features in one row
                book_parse = pd.DataFrame({'Name': book_name, 'Pub_info': book_pubinfo, 
                                           'Date': book_date, 'Star': book_star, 
                                           'Tag': book_tag, 'Comment': book_comment, 
                                           'Website': book_website}, 
                                           index=[0])
                
                # concat the book info together
                page_data = pd.concat([page_data, book_parse], ignore_index = True)
                
            except:
                
                # print 'error' if wrong
                print(book_name, 'parse error')
                
    return page_data

In [4]:
# create the data for all dataframe
data = pd.DataFrame(columns = ['Name', 'Pub_info', 'Date', 'Star', 'Tag', 'Comment', 'Website'])

# get the number of books
url = f'https://book.douban.com/people/{name}/collect?start=0&sort=time&rating=all&filter=all&mode=grid'
web_data = requests.get(url, headers = header)
soup = BeautifulSoup(web_data.text, features="lxml")

web_title = str(soup.title).split('>')[1].split('<')[0].replace('  ','').replace('\n','')
num_books = int(re.findall(r"\(\s*\+?(-?\d+)\s*\)", web_title)[0])
print('Total book #:', num_books)

# parse the book information by page
for i in range(0, math.ceil(num_books / 15)+1): # num_books ange(0, math.ceil(num_books / 15)+1)
    if i != math.ceil(num_books / 15):
        print(f'Parsing books: {i*15+1} ~ {i*15+15}...')
    else:
        print(f'Parsing books: {i*15+1} ~ {num_books}...')
    url = f'https://book.douban.com/people/{name}/collect?start={i*15}&sort=time&rating=all&filter=all&mode=grid'
    book_info = get_books_info_from_page(url, header)
    data = pd.concat([data, book_info], ignore_index = True)
    
print('Finish parsing all books.')

Total book #: 489
Parsing books: 1 ~ 15...
Parsing books: 16 ~ 30...
Parsing books: 31 ~ 45...
Parsing books: 46 ~ 60...
Parsing books: 61 ~ 75...
Parsing books: 76 ~ 90...
Parsing books: 91 ~ 105...
Parsing books: 106 ~ 120...
Parsing books: 121 ~ 135...
Parsing books: 136 ~ 150...
Parsing books: 151 ~ 165...
Parsing books: 166 ~ 180...
Parsing books: 181 ~ 195...
Parsing books: 196 ~ 210...
Parsing books: 211 ~ 225...
Parsing books: 226 ~ 240...
Parsing books: 241 ~ 255...
Parsing books: 256 ~ 270...
Parsing books: 271 ~ 285...
Parsing books: 286 ~ 300...
Parsing books: 301 ~ 315...
Parsing books: 316 ~ 330...
Parsing books: 331 ~ 345...
Parsing books: 346 ~ 360...
Parsing books: 361 ~ 375...
Parsing books: 376 ~ 390...
Parsing books: 391 ~ 405...
Parsing books: 406 ~ 420...
Parsing books: 421 ~ 435...
Parsing books: 436 ~ 450...
Parsing books: 451 ~ 465...
Parsing books: 466 ~ 480...
Parsing books: 481 ~ 495...
Parsing books: 496 ~ 489...
Finish parsing all books.


In [7]:
# clean the final parsed data
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date').reset_index(drop=True)
data

,Name,Pub_info,Date,Star,Tag,Comment,Website
0,瓦尔登湖,[美] 梭罗 / 王家湘 / 北京十月文艺出版社 / 2009-1 / 28.00元,2012-04-08,5,哲学 英语文学 外国文学,"神圣的湖,寂寞的人,孤独的书",https://book.douban.com/subject/3522695/
1,神秘岛（上、下）,儒勒·凡尔纳 / 文思 / 上海百家出版社 / 2009-1 / 40.00元,2012-04-08,4,外国文学 法语文学 科幻小说,类似《鲁滨逊漂流记》，但主角们造出了许多高科技事物。,https://book.douban.com/subject/3527769/
2,译边草,周克希 / 上海三联书店 / 2008-11 / 25.00元,2012-04-08,5,汉语文学 文学评论 #三联书店,,https://book.douban.com/subject/3364275/
3,追寻逝去的时光,[法] 马塞尔·普鲁斯特 原著、[法] 斯泰凡·厄埃、[法] 斯丹尼斯拉·布雷泽 改编、[法...,2012-04-08,4,#人民文学 绘本 外国文学 普鲁斯特,,https://book.douban.com/subject/1796747/
4,在少女花影下,[法] 马塞尔·普鲁斯特 著、[荷] 凡·东恩 绘 / 周克希 / 人民文学出版社 / 20...,2012-04-08,5,#人民文学 法语文学 外国文学 普鲁斯特,,https://book.douban.com/subject/4199421/
...,...,...,...,...,...,...,...
484,安娜·卡列尼娜,[俄] 列夫·托尔斯泰 / 靳戈 / 上海译文出版社 / 2021-6 / 118元,2025-03-12,4,外国文学 俄语文学 #上海译文,这次有机会重读，因为一些其他事，读得断断续续的。常常读着读着就忘记了前面。不过总能从托翁的文...,https://book.douban.com/subject/35514567/
485,守望之心,[美] 哈珀·李 / 张芸 / 译林出版社 / 2017-2 / 48.00元,2025-03-12,4,外国文学 英语文学,“偏见是一个贬义词，信念是一个褒义词，这两者之间具有某些共性：它们的起始都是理性的终结。”如...,https://book.douban.com/subject/26909081/
486,涅朵奇卡,陀思妥耶夫斯基 / 于大卫 / 山东文艺出版社 / 2023-9 / 45.00元,2025-04-01,4,外国文学 俄语文学,如果不是读过《白夜》，不会相信这么纤细的一本书是由那个大胡子，写过《罪与罚》和《卡拉马佐夫兄...,https://book.douban.com/subject/36504167/
487,65种微习惯,[日本] 古川武士 / 何俊山何巴特译 / 人民邮电出版社 / 2022-1 / 59.80,2025-04-07,4,应用书籍,因为一直负面思考，所以我活得很不快乐，也让周围的人觉得很累。我想下定决心改变自己，所以就读了...,https://book.douban.com/subject/35768143/


In [8]:
# export the data to a excel file
data.to_excel("Douban_books_result.xlsx")